## 라이브러리 및 함수 선언

In [1]:
import random

import pandas as pd
import numpy as np
import h5py
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model

2022-10-13 01:27:50.716927: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def weighted_mape(y_true, y_pred):
    tot = tf.reduce_sum(y_true)
    # 텐서 안의 값이 지정된 범위를 넘지 않게 해줌
    tot = tf.clip_by_value(tot, clip_value_min=0.01, clip_value_max=10000000000000) 
    wmape = tf.realdiv(tf.reduce_sum(tf.abs(tf.subtract(y_true,y_pred))), tot)*100
    
    return wmape

In [3]:
def mape(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

## 검증 데이터셋 불러오기

In [4]:
with h5py.File('../dataset/train_test_data.hdf5', 'r') as hf:
    X_test = hf['x_test'][:]
    Y_test = hf['y_test'][:]

X_test.shape, Y_test.shape

((251087, 107), (251087,))

## 모델 정의

In [5]:
def flow_prediction_model(x_size, y_size, learning_rate, dropout_ratio, epsilon):
    model = Sequential()
    
    model.add(Dense(512, activation='relu', input_shape=(x_size, ), kernel_initializer='RandomNormal'))
    Dropout(dropout_ratio)
    model.add(Dense(256, activation='relu'))
    Dropout(dropout_ratio)
    model.add(Dense(128, activation='relu'))
    Dropout(dropout_ratio)
    model.add(Dense(64, activation='relu'))
    Dropout(dropout_ratio)
    model.add(Dense(32, activation='relu'))
    Dropout(dropout_ratio)
    model.add(Dense(16, activation='relu'))
    Dropout(dropout_ratio)
    model.add(Dense(8, activation='relu'))
    Dropout(dropout_ratio)
    model.add(Dense(y_size))
    
    model.compile(loss=weighted_mape,
                  optimizer=Adam(learning_rate=learning_rate, epsilon=epsilon),
                  metrics=['mae'])
    
    return model

In [6]:
# Hyperparameter
learning_rate = 0.0001
dropout_ratio = 0.3
epsilon = 0.001  # Batch normalization

model = flow_prediction_model(x_size=X_test.shape[1], y_size=1, learning_rate=learning_rate,
                              dropout_ratio=dropout_ratio, epsilon=epsilon)
model.summary()

2022-10-13 01:27:51.975231: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-13 01:27:52.053877: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 01:27:52.054673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-10-13 01:27:52.054792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-13 01:27:52.055531: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               55296     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 1

## 모델 가중치 불러오기

In [7]:
train_model = load_model('../weight/flow_prediction.h5', custom_objects={'weighted_mape': weighted_mape})
train_model

## 모델 성능 평가 (Mean Absolute Percentage Error) - 5회 반복

In [8]:
# 랜덤 인덱스 생성
dataset_index_list = list(range(X_test.shape[0]))

random_index_list = random.sample(dataset_index_list, 100)
random_index_list

[220504,
 83677,
 100045,
 47841,
 245932,
 86796,
 84177,
 215901,
 38383,
 136549,
 11080,
 180139,
 134198,
 105702,
 121938,
 144378,
 206669,
 14064,
 22951,
 208095,
 88744,
 132067,
 49136,
 85104,
 193395,
 83767,
 59876,
 235270,
 244481,
 92095,
 115864,
 110115,
 5649,
 74024,
 232310,
 135000,
 207940,
 104749,
 126276,
 5121,
 4485,
 222082,
 225915,
 82919,
 40402,
 51878,
 201061,
 144944,
 18045,
 122566,
 159455,
 246035,
 78482,
 110175,
 6219,
 153988,
 53499,
 231335,
 243044,
 69581,
 151064,
 147599,
 216527,
 105177,
 91999,
 168021,
 206656,
 101798,
 32047,
 10403,
 55657,
 47131,
 28457,
 201598,
 27385,
 220530,
 228972,
 150238,
 87412,
 43249,
 99729,
 191431,
 152128,
 212460,
 208421,
 39110,
 34791,
 209839,
 60255,
 196855,
 32868,
 48252,
 37787,
 104983,
 195226,
 26902,
 110848,
 138660,
 154839,
 66197]

In [9]:
test_num = 5
mape_list = []

dataset_index_list = list(range(X_test.shape[0]))


for test_idx in range(test_num):
    random_index_list = random.sample(dataset_index_list, 100)

    X_test_extraction = X_test[random_index_list]
    Y_test_extraction = Y_test[random_index_list]

    predictions = train_model.predict(X_test_extraction).reshape(-1)
    flow_prediction_mape = mape(Y_test_extraction, predictions)

    mape_list.append(flow_prediction_mape)
    
    print("[Test {}] 음원 흐름 예측 모델 MAPE: {:.4f}".format(test_idx+1, flow_prediction_mape))

print("="*50)
print("음원 흐름 예측 모델 평균 MAPE: {:.4f}".format(np.mean(mape_list)))
print("="*50)

2022-10-13 01:27:55.531625: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


[Test 1] 음원 흐름 예측 모델 MAPE: 3.8100
[Test 2] 음원 흐름 예측 모델 MAPE: 3.2138
[Test 3] 음원 흐름 예측 모델 MAPE: 4.2542
[Test 4] 음원 흐름 예측 모델 MAPE: 3.2348
[Test 5] 음원 흐름 예측 모델 MAPE: 4.4477
음원 흐름 예측 모델 평균 MAPE: 3.7921
